In [76]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

import numpy as np
import pandas as pd 
from BHDVCS_torch import TBHDVCS

import BHDVCS_fit as dvcsfit

import matplotlib
import matplotlib.pyplot as plt

from scipy.stats import chisquare

import time
import sys

In [85]:
tb = TBHDVCS()

f = tb.TotalUUXS_curve_fit3
loss_func = tb.loss_MSE

In [86]:
dats = pd.read_csv('dvcs_psuedo.csv')
n = np.array(dats['#Set'])
ind = np.array(dats['index'])
k = np.array(dats['k'])
qq = np.array(dats['QQ'])
xb = np.array(dats['x_b'])
t = np.array(dats['t'])
phi = np.array(dats['phi_x'])
ydat = np.array(dats['F'])
errF = np.array(dats['errF']) 
F1 = np.array(dats['F1'])
F2 = np.array(dats['F2'])
const = np.array(dats['dvcs'])  
ReH_target = np.array(dats['ReH']) 
ReE_target = np.array(dats['ReE']) 
ReHT_target = np.array(dats['ReHtilde'])
err_H = []
err_E = []
err_HT = []

In [87]:
epoch_ops = [500, 1000, 1500, 2000, 2500]

In [88]:
node_ops = [10, 20, 50, 75, 100]

In [89]:
learning_rate_ops = [0.001, 0.01, 0.05, 0.1, 0.5, 1, 5, 10]

In [102]:
def reset_nets(nodes):
    net1 = torch.nn.Sequential(
            torch.nn.Linear(4, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, 3)
        )

    net2 = torch.nn.Sequential(
            torch.nn.Linear(4, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, 3)
        )

    net3 = torch.nn.Sequential(
            torch.nn.Linear(4, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, 3)
        )

    net4 = torch.nn.Sequential(
            torch.nn.Linear(4, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, 3)
        )

    net5 = torch.nn.Sequential(
            torch.nn.Linear(4, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, 3)
        )

    net6 = torch.nn.Sequential(
            torch.nn.Linear(4, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, 3)
        )

    net7 = torch.nn.Sequential(
            torch.nn.Linear(4, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, 3)
        )

    net8 = torch.nn.Sequential(
            torch.nn.Linear(4, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, nodes),
            torch.nn.Tanh(),
            torch.nn.Linear(nodes, 3)
        )

    return [net1, net3, net5, net8]

In [103]:
def train(network, eps):
    for epoch in range(eps):
            preds = network(x.float()) #output 3 predicted values for cffs

            ReHfit = torch.mean(torch.transpose(preds, 0, 1)[0])
            ReEfit = torch.mean(torch.transpose(preds, 0, 1)[1])
            ReHTfit = torch.mean(torch.transpose(preds, 0, 1)[2])
            cffs = [ReHfit, ReEfit, ReHTfit]

            loss = loss_func((xdat.float()), cffs, errs, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    return preds

In [106]:
for datset in range(13):
    start_time = time.time()
    best = 100
    step = 0
    a = datset*36
    b = a + 36


    xdat = np.asarray([phi[a:b], qq[a:b], xb[a:b], t[a:b], k[a:b], F1[a:b], F2[a:b], const[a:b]])
    x = Variable(torch.from_numpy(xdat[1:5].transpose()))
    y = Variable(torch.from_numpy(ydat[a:b].transpose()))
    xdat = Variable(torch.from_numpy(xdat))
    errs = Variable(torch.from_numpy(errF[a:b]))
    
    for e in epoch_ops:
        for n in node_ops:
            net_ops = reset_nets(n)
            for l in learning_rate_ops:
                layers = 0
                for net in net_ops:
                    layers+=1
                    step+=1
                    
                    optimizer = torch.optim.Adam(net.parameters(), lr=l)
                    p = train(net, e)
                    ReHfit = torch.mean(torch.transpose(p, 0, 1)[0]).data.numpy()
                    ReEfit = torch.mean(torch.transpose(p, 0, 1)[1]).data.numpy()
                    ReHTfit = torch.mean(torch.transpose(p, 0, 1)[2]).data.numpy()
                    fit_cffs = [ReHfit, ReEfit, ReHTfit]
                    
                    err_H = abs(100*((fit_cffs[0]-ReH_target[a]))/ReH_target[a])
                    err_E = abs(100*((fit_cffs[1]-ReE_target[a]))/ReE_target[a])
                    err_HT = abs(100*((fit_cffs[2]-ReHT_target[a]))/ReHT_target[a])
                    
                    avgErr = (err_H + err_E + err_HT)/3
                    
                    if avgErr < best:
                        best = avgErr
                        optims = [layers, l, n, e]
                    
#                     tot_steps = len(epoch_ops)*len(node_ops)*len(learning_rate_ops)*len(net_ops)
#                     pct_done = 100*(step/tot_steps)*(step/tot_steps)
#                     mins = (time.time() - start_time)/60
#                     sys.stdout.write("\rOptimizing set %d: Step %d of %d, %.2f%% complete. %.0f of %.0f minutes." % (datset, step, tot_steps, pct_done, mins, (100/pct_done)*mins))
#                     sys.stdout.flush()
                        
    print('Optimized set #%d to %.2f%%' % (datset, best))
#     print('Optimization of set %d took %.1f minutes.\n' % (datset, mins))
    print('Optimizal architecture for set %d: %d layers, %f learning rate, %d nodes, %d e epochs.' % (datset, optims[0], optims[1], optims[2], optims[3]))

Optimizing set 0: Step 800 of 800, 100.00% complete. 102 of 102 minutes.Optimized set #0 to -59.74%
Optimization of set 0 took 102.0 minutes.

Optimizal architecture for set 0: 3 layers, 10.000000 learning rate, 100 nodes, 500 e epochs.
Optimizing set 1: Step 275 of 800, 11.82% complete. 16 of 134 minutes.

KeyboardInterrupt: 